In [13]:
import urllib.request
from urllib.request import urlopen
import pandas as pd
import geojson
import matplotlib.pyplot as plt 


In [7]:
url_cgg_code = 'https://openprescribing.net/api/1.0/org_details/?org_type=ccg&keys=total_list_size&format=json'
ccg_code_response = urllib.request.urlopen(url_cgg_code )
ccg_code_data = json.loads(ccg_code_response.read())
ccg_code_flat_data = pd.json_normalize(ccg_code_data)
ccg_codes_1 = ccg_code_flat_data.drop_duplicates(['row_id'])
ccg_codes_2 = ccg_codes_1.drop(columns = {"date", "row_name", "total_list_size"})
ccg_codes_list = ccg_codes_2['row_id'].to_list()


In [ ]:
gp_locations = []

for ccg_code in ccg_codes_list:
    url_gp_location = ("https://openprescribing.net/api/1.0/org_location/?q="
    + str(ccg_code)
    + '&format=json')
    response = urllib.request.urlopen(url_gp_location)
    data = geojson.loads(response.read())
    data_2 = list(data.values())
    gp_locations.append(data_2)

In [19]:
data

{"crs": {"properties": {"name": "EPSG:4326"}, "type": "name"}, "features": [{"geometry": {"coordinates": [-1.713164, 53.722207], "type": "Point"}, "properties": {"code": "B85001", "name": "PARKVIEW SURGERY", "setting": 4}, "type": "Feature"}, {"geometry": {"coordinates": [-1.626482, 53.588349], "type": "Point"}, "properties": {"code": "B85002", "name": "DEARNE VALLEY HEALTH CENTRE", "setting": 4}, "type": "Feature"}, {"geometry": {"coordinates": [-1.634884, 53.689744], "type": "Point"}, "properties": {"code": "B85004", "name": "CALDER VIEW SURGERY", "setting": 4}, "type": "Feature"}, {"geometry": {"coordinates": [-1.7122, 53.585727], "type": "Point"}, "properties": {"code": "B85005", "name": "SHEPLEY HEALTH CENTRE", "setting": 4}, "type": "Feature"}, {"geometry": {"coordinates": [-1.783309, 53.578363], "type": "Point"}, "properties": {"code": "B85006", "name": "ELMWOOD FAMILY DOCTORS", "setting": 4}, "type": "Feature"}, {"geometry": {"coordinates": [-1.636481, 53.715329], "type": "Poin

In [14]:
gp_locations = []

for ccg_code in ccg_codes_list:
    url_gp_location = ("https://openprescribing.net/api/1.0/org_location/?q="
    + str(ccg_code)
    + '&format=json')
    response = urllib.request.urlopen(url_gp_location)
    data = geojson.loads(response.read())
    data_2 = list(data.values())
    gp_locations.append(data_2)
